In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps
from SimPEG import Utils
from SimPEG.Utils import mkvc
import SimPEG.PF as PF
import scipy as sp
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
workDir = ''
fName = 'DataGrid.tiff'

dataGrid = DataIO.loadGeoTiffFile(workDir+fName, plotIt=True)

## Invert the data


In [2]:
Simulator.ViewMagSurveyWidget(dataGrid)

interactive(children=(FloatSlider(value=-10.036722527700476, continuous_update=False, description='East', max=2539.9632774722995, min=-2560.0367225277005, step=10.0), FloatSlider(value=-28.5773809524253, continuous_update=False, description='North', max=2771.4226190475747, min=-2828.5773809524253, step=10.0), FloatSlider(value=90.0, continuous_update=False, description='Azimuth', max=180.0, step=5.0), FloatSlider(value=1893.5746618499097, continuous_update=False, description='Length', max=3787.1493236998194, min=20.0, step=20.0), BoundedFloatText(value=100.0, description='Sampling', max=1000.0, min=10.0, step=5.0), Output()), _dom_classes=('widget-interact',))

In [3]:
# Wrap the inversion inside a function
xLoc = np.asarray(range(dataGrid.nx))*dataGrid.dx+dataGrid.limits[0]
yLoc = np.asarray(range(dataGrid.ny))*dataGrid.dy+dataGrid.limits[2]
XX,YY = np.meshgrid(xLoc,yLoc)

xyzLoc = np.c_[mkvc(XX), mkvc(YY), mkvc(XX**0.)*40.]

topo = xyzLoc.copy()
topo[:,2] -= 40


# I want 10 m cells in x, y and z covering our survey
h = np.r_[50., 50., 50.]

# Also need to paddout far enough in case there is a regional field
# Here I am sending distances N, S, E, W, above and below all at once
# Let's say 200 m padding
padDist = np.ones((3,2))*1000.

mesh = ProblemSetter.meshBuilder(xyzLoc, h, padDist,
                   meshGlobal=None,
                   expFact=1.3,
                   meshType='TREE',
                   verticalAlignment='center')

ProblemSetter.refineTree(mesh, xyzLoc, finalize=True, dtype="surface", nCpad=[3, 3, 3])

actv = Utils.modelutils.surface2ind_topo(mesh, topo, gridLoc='CC', method='nearest')
nC = int(np.sum(actv))

AttributeError: 'TreeMesh' object has no attribute 'insert_cells'

In [ ]:


H = [50000, 90, 0]

# Create the survey
rx = PF.BaseMag.RxObs(xyzLoc)
srcField = PF.BaseMag.SrcField([rx], param=H)
survey = PF.BaseMag.LinearSurvey(srcField)
survey.dobs = mkvc(dataGrid.values)

# Creat reduced identity map
idenMap = Maps.IdentityMap(nP=3*nC)

# Create the forward model operator
prob = PF.Magnetics.MagneticVector(mesh, chiMap=idenMap,
                                     actInd=actv)
# Pair the survey and problem
survey.pair(prob)

In [ ]:
from SimPEG import Regularization, DataMisfit, Inversion, Directives, Optimization, InvProblem
# This Mapping connects all the regularizations together
wires = Maps.Wires(('p', nC), ('s', nC), ('t', nC))

# Create sensitivity weights from our linear forward operator
# so that all cells get equal chance to contribute to the solution
wr = np.sum(prob.G**2., axis=0)**0.5
wr = (wr/np.max(wr))


# Create three regularization for the different components
# of magnetization
reg_p = Regularization.Sparse(mesh, indActive=actv, mapping=wires.p)
reg_p.cell_weights = (wires.p * wr)
reg_p.mref = np.zeros(3*nC)

reg_s = Regularization.Sparse(mesh, indActive=actv, mapping=wires.s)
reg_s.cell_weights = (wires.s * wr)
reg_s.mref = np.zeros(3*nC)

reg_t = Regularization.Sparse(mesh, indActive=actv, mapping=wires.t)
reg_t.cell_weights = (wires.t * wr)
reg_t.mref = np.zeros(3*nC)

reg = reg_p + reg_s + reg_t
reg.mref = np.zeros(3*nC)

# Data misfit function
dmis = DataMisfit.l2_DataMisfit(survey)
dmis.W = survey.dobs**0. * 1e-1

# Choose a solver
opt = Optimization.ProjectedGNCG(maxIter=7, lower=-10., upper=10.,
                                 maxIterCG=20, tolCG=1e-3)

# The inverse problem needs to how the misfit, regularizer and solver
invProb = InvProblem.BaseInvProblem(dmis, reg, opt)

# Add directives to the inversion
betaest = Directives.BetaEstimate_ByEig()

# Here is where the norms are applied usually
IRLS = Directives.Update_IRLS(f_min_change=1e-3,
                              minGNiter=1)

update_Jacobi = Directives.UpdateJacobiPrecond()
targetMisfit = Directives.TargetMisfit()

# Create active map to go from reduce set to full
actvMap = Maps.InjectActiveCells(mesh, actv, 0)
# saveModel = Directives.SaveUBCModelEveryIteration(mapping=actvMap)
# saveModel.fileName = 'MVI_C'

# Connect all the pieces together
inv = Inversion.BaseInversion(invProb,
                              directiveList=[betaest, IRLS, update_Jacobi,
                                            ])

# Invert with a starting model
mstart = np.ones(3*nC) * 1e-4
mrec_MVI = inv.run(mstart)

In [ ]:
np.mean([mesh.gridCC[:,1].max(), mesh.gridCC[:,1].min()])

In [ ]:
fileName = 'Recovered_Model_l2Norm.png'
actvMap = Maps.InjectActiveCells(mesh, actv, np.nan)
fig, axs = plt.figure(), plt.subplot()
amp = np.sum(invProb.model.reshape((nC,3),order='F')**2., axis=1)**0.5
mesh.plotSlice(actvMap*amp, normal='Y', ind=65, ax=axs)

axs.set_xlim([-18000, -16000])
axs.set_ylim([-1500, 0])
axs.set_aspect('equal')

In [ ]:
mesh.h_gridded